C Conversion
---
All we need to do to convert our Mahalanobis Distance model to C is to save the mean vector and inverse of the covariance matrix as constant arrays.

In [1]:
from os import listdir
from os.path import join
import numpy as np
import scipy as sp
from scipy import stats
import c_writer

In [2]:
# Print versions
!python --version
print('Numpy ' + np.__version__)
print('SciPy ' + sp.__version__)

Python 3.7.6
Numpy 1.18.1
SciPy 1.4.1


In [3]:
# Settings (define file locations)
models_path = 'models'  # Where we can find the model files (relative path location)
md_file_name = 'md_model'   # Mahalanobis Distance model arrays (.npz will be added)
c_model_name = 'md_model'   # Will be given .h suffix

c_mean_name = 'model_mu'
c_inv_cov_name = 'model_inv_cov'

In [4]:
# Load model
with np.load(join(models_path, md_file_name) + '.npz') as data:
    model_mu = data['model_mu']
    model_cov = data['model_cov']
print(model_mu)
print(model_cov)

[0.01204995 0.00709583 0.00497362]
[[5.36191246e-07 8.13786731e-08 3.46141514e-08]
 [8.13786731e-08 4.76039917e-07 1.12038078e-08]
 [3.46141514e-08 1.12038078e-08 3.15563520e-07]]


In [5]:
# Calculate inverse of covariance matrix (as it's constant)
inv_cov = sp.linalg.inv(model_cov)
print(inv_cov)

[[1927196.25782373 -324748.88969416 -199864.163168  ]
 [-324748.88969416 2157143.90610544  -40965.82067725]
 [-199864.163168    -40965.82067725 3192311.64978127]]


In [6]:
# Create constant C arrays for model (mu and inverse covariance)
c_mu = c_writer.create_array(model_mu, 'float', c_mean_name)
c_inv_cov = c_writer.create_array(inv_cov, 'float', c_inv_cov_name)

In [7]:
# Construct header file
header_str = c_writer.create_header(c_mu + '\n' + c_inv_cov, c_model_name)
print(header_str)

#ifndef MD_MODEL_H
#define MD_MODEL_H

const unsigned int model_mu_dim1 = 3;

const float model_mu[3] = {
    0.012049953994017095, 0.00709583131025641, 0.004973618240170936
};

const unsigned int model_inv_cov_dim1 = 3;
const unsigned int model_inv_cov_dim2 = 3;

const float model_inv_cov[3][3] = {
    1927196.2578237287, -324748.88969416154, -199864.16316800224, 
    -324748.88969416154, 2157143.906105444, -40965.82067724897, 
    -199864.16316800224, -40965.82067724897, 3192311.649781269
};

#endif //MD_MODEL_H


In [8]:
# Save C header file
with open(join(models_path, c_model_name) + '.h', 'w') as file:
    file.write(header_str)

Save Samples for Testing
---
Convert raw sample to constant C array for testing.

In [54]:
# Saved Numpy test samples file location
sample_file_path = 'test_samples'
sample_file_name = 'normal_anomaly_samples'  # Will be given .npz suffix
c_normal_sample_name = 'normal_sample'       # Will be given .h suffix for file
c_anomaly_sample_name = 'anomaly_sample'     # Will be given .h suffix for file

In [55]:
# Load test samples
with np.load(join(sample_file_path, sample_file_name) + '.npz') as data:
    normal_sample = data['normal_sample']
    anomaly_sample = data['anomaly_sample']
print(normal_sample.shape)
print(anomaly_sample.shape)
print(normal_sample[:5])

(200, 3)
(200, 3)
[[-0.004395  0.208008  0.975586]
 [-0.007324  0.203125  0.979981]
 [-0.01416   0.200195  0.978027]
 [-0.002441  0.21582   0.979981]
 [-0.022461  0.19873   0.977051]]


In [57]:
# Create C array out of normal sample
c_sample = c_writer.create_array(normal_sample, 'float', c_normal_sample_name)
header_str = c_writer.create_header(c_sample, c_normal_sample_name)
#print(header_str)

In [12]:
# Save C header file with normal sample
with open(join(sample_file_path, c_normal_sample_name) + '.h', 'w') as file:
    file.write(header_str)

In [58]:
# Create C array out of anomaly sample
c_sample = c_writer.create_array(anomaly_sample, 'float', c_anomaly_sample_name)
header_str = c_writer.create_header(c_sample, c_anomaly_sample_name)
#print(header_str)

In [59]:
# Save C header file with normal sample
with open(join(sample_file_path, c_anomaly_sample_name) + '.h', 'w') as file:
    file.write(header_str)

Develop C functions
---
We don't have access to an easy/efficient library for calculating the Mahalanobis Distance in C, so we need to write a few functions from scratch. It's easiest to get the algorithms down in Python first so we can compare the output from a few samples to known good algorithms.

In [13]:
# Set some settings to test out our functions
sensor_sample_rate = 200    # Hz
sample_time = 0.64           # Time (sec) length of each sample
max_measurements = int(sample_time * sensor_sample_rate)

In [14]:
# Test extracting features (median absolute deviation) using SciPy
sample = normal_sample[0:max_measurements]                  # Truncate to 128 measurements
sp_test_features = stats.median_absolute_deviation(sample)  # Calculate MAD
print(sp_test_features)

[0.01158355 0.00651454 0.00434402]


In [46]:
# Find median in an array (we'll use qsort in C)
# We want to calculate MAD in a C-like fashion to make it easier to port
def median(arr):
    n = len(arr)
    sorted_arr = np.sort(arr)
    if n % 2 == 0:
        median = (sorted_arr[int((n - 1) / 2)] + sorted_arr[int(n / 2)]) / 2.0
    else:
        median = sorted_arr[n / 2]
    return median

In [47]:
# Calculate MAD manually
def calc_mad(arr):
    
    # Find number of elements
    n = len(arr)
    
    # Get median of array
    med = median(arr)

    # Calculate absolute deviation from median for each element
    devs = [0] * n
    for i in range(n):
        devs[i] = abs(arr[i] - med)
    
    # Find median of deviations
    return median(devs)

In [48]:
# Wrapper to extract MAD features for all axis
def extract_mad_features(sample, scale=1.4826):
    mads = []
    
    # Calculate MAD for each axis (note: SciPy scales by 1.4826)
    # https://github.com/scipy/scipy/issues/11090
    for axis in range(sample.shape[1]):
        mads.append(scale * calc_mad(sample[:, axis]))
        
    return mads

In [51]:
# Test MAD algorithm and compare to SciPy answer
sample = normal_sample[0:max_measurements]                  # Truncate to 128 measurements
normal_x = extract_mad_features(sample)
print(normal_x)

[0.0115835538, 0.006514544399999972, 0.0043440179999999825]


In [52]:
# Test: calculate Mahalanobis distance using Numpy
x_minus_mu = normal_x - model_mu
left_term = np.dot(x_minus_mu, inv_cov)
mahal = np.dot(left_term, x_minus_mu.T)
print(mahal)

2.0900788347195736


In [20]:
# Calculate the dot product of two vectors
def dot_product(a, b):
    sum = 0
    for i in range(len(a)):
        sum += a[i] * b[i]
        
    return sum

In [60]:
# Do matrix multiplication
def matrix_multiply(a, b):
    
    # Find number of rows and columns
    a_rows = len(a)
    a_cols = len(a[0])
    b_rows = len(b)
    b_cols = len(b[0])
    
    # Check to make sure we can multiply the matrices
    if a_cols != b_rows:
        print('Dimension mismatch')
        return [[]]
    
    # Create return matrix
    prod = [[0 for i in range(b_cols)] for j in range(a_rows)]
    
    # Calculate the dot product for each element in the return matrix
    for i in range(a_rows):
        for j in range(b_cols):
            for k in range(a_cols):
                prod[i][j] += a[i][k] * b[k][j]
    
    return prod

In [63]:
# Calculate Mahalanobis distance the C way (with const inverse covariance)
def mahalanobis(x, mu, inv_cov):
    
    # Subtract each element in X from the mean
    x_minus_mu = [0 for i in range(len(x))]
    for i in range(len(x)):
        x_minus_mu[i] = x[i] - mu[i]
        
    # Compute product of prev term and inverse covariance
    left_term = matrix_multiply([x_minus_mu], inv_cov)
    
    # Transpose difference matrix
    x_minus_mu_t = [[i] for i in x_minus_mu]
    
    # Matrix multiply prev term and difference
    mahal = matrix_multiply(left_term, x_minus_mu_t)
    
    return mahal[0][0]

In [64]:
# Test Mahalanobis function with normal sample
mahalanobis(normal_x, model_mu.tolist(), inv_cov.tolist())

2.0900788347195736

In [65]:
# Find MAD for anomaly sample
sample = anomaly_sample[0:max_measurements]  # Truncate to 128 measurements
anomaly_x = extract_mad_features(sample)
mahalanobis(anomaly_x, model_mu, inv_cov)

4844.40534566512